# Volume 3: Sklearn Guide
    <Name>
    <Class>
    <Date>

## Problem 1

Take your Naive Bayes classifier from your homework and rewrite it as a class that inherits from `BaseEstimator` and `ClassifierMixin`.
Implement `__init__()`, `fit()`, and `predict()` in a way that matches `sklearn` conventions.

Test your model on the iris dataset.

## Problem 2

Write a transformer class where the `fit()` and `transform()` methods takes in $X$ as a pandas Data Frame.
For each numerical column, replace any `nan` entries with the mean of the column.
Drop string columns.
Return the data as a NumPy array.

## Problem 3

Use `cross_validate()` to score your class from Problem 1 on the iris dataset.
Do the same for a `LogisticRegressionClassifier`.

## Problem 4

Take the cancer data set (`datasets.load_breast_cancer()`) and do a grid search on an SVM (`sklearn.linear.svm`) with the parameter `C` as .01, .1, or 1, and the parameter `kernel` as `"linear"`, `"poly"`, `"rbf"`, and `"sigmoid"`.

What is the best choice of parameters?
How well does the corresponding model do?

## Problem 5

Make a pipeline of your transformer from Problem 2, a normalizing scaler transformer (`preprocessing.StandardScaler`), a PCA transformer (`decomposition.PCA`), and an SVM classifier (`svm.SVC`).
Using the titanic dataset (read in as a pandas DataFrame), do a grid search for the best model, varying your parameters however you see fit.

What is your best choice of parameters?
How well does the corresponding model do?

**Extra credit** to the student with the very best model!
To compete, pick your best parameters, do a cross validation with 10 folds, and take the average of the test scores.